In [8]:
import pandas as pd
import numpy as np
import random

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
class Node:
    def __init__(self, n_inputs):
        self.weights = [random.random() for i in range(n_inputs)]
        self.inputs = []
        self.output = 0
        self.error = 0

In [ ]:
network = [[Node(1) for i in range(5)],
          [Node(5) for i in range(7)],
          [Node(7) for i in range(5)],
          [Node(5) for i in range(1)]]

In [ ]:
def forward_propogate(network, data):
    x = []
    for i in range(len(data)):
        network[0][i].inputs.append(data[i])
        network[0][i].output = sigmoid(network[0][i].inputs[0]*network[0][i].weights[0])
        x.append(network[0][i].output)
    for i in range(1,len(network)-1):
        l = []
        for j in range(len(x)):
            network[i][j].inputs = x
            s = 0
            for k in range(len(x)):
                s += network[i][j].inputs[k]*network[i][j].weights[k]
            network[i][j].output = sigmoid(s)
            l.append(network[i][j].output)
        x = l
    return network


In [ ]:
def error(actual,expected):
    return [actual[i]-expected[i] for i in range(len(actual))]

In [ ]:
def back_propogate(network,error,lr = 0.001):
    for j in range(2):
        network[-1][j].error = error[j]*(network[-1][j].output)*(1-network[-1][j].output)
        s = [network[-1][j].error*network[-1][j].weights[i] for i in range(len(network[-1][j].inputs))]
        network[-1][j].weights = [network[-1][j].weights[i] + lr*network[-1][j].error*network[-1][j].inputs[i] for i in range(len(network[-1][j].weights))]
        for i in range(len(network[-2])):
            network[-2][i].error += s[i]
    
    for i in range(len(network)-2,-1,-1):
        for j in range(len(network[i])):
            t = network[i][j].error*network[i][j].output*(1-network[i][j].output)
            w = network[i][j].weights
            for k in range(len(network[i][j].weights)):
                network[i][j].weights[k] += t*network[i][j].inputs[k]
            if i!=0:
                for k in range(len(network[i-1])):
                    network[i-1][k].error += network[i][j].error*w[k]
    return network

In [1]:
def select_paths_r(network,n):
    x = []
    for i in range(n):
        y = []
        for j in range(len(network)):
            y.append(random.randint()%len(network[j]))
        x.append(y)
    return x

In [4]:
def crossover(p1,p2,x):
    pn1 = p1[:x]+p[x:]
    pn2 = p2[:x]+p1[x:]
    return pn1,pn2

In [5]:
def mutate(ppln,strand,m_prob= 0.1):
    if random.random()<m_prob:
        x = random.randint()%len(strand)
        mstrand = strand[:x]+[ppln[x][random.randint()%len(ppln[x])]]+[strand[x+1:]]
        return mstrand
    return -1

In [9]:
data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')

from sklearn.preprocessing import MinMaxScaler
 
mx = MinMaxScaler()
cols = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education']
data[cols] = mx.fit_transform(data[cols])

data.drop(["ID","Mortgage","Personal Loan", "Securities Account","CD Account","Online","ZIP Code"],axis=1,inplace=True)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Age         5000 non-null   float64
 1   Experience  5000 non-null   float64
 2   Income      5000 non-null   float64
 3   Family      5000 non-null   float64
 4   CCAvg       5000 non-null   float64
 5   Education   5000 non-null   float64
 6   CreditCard  5000 non-null   int64  
dtypes: float64(6), int64(1)
memory usage: 273.6 KB


In [10]:
x,y = data.iloc[:,:-1],data.iloc[:,-1]

In [13]:
from sklearn.model_selection import train_test_split

xtr,xt,ytr,yt = train_test_split(x,y,test_size = 0.3)

xtr,xo,ytr,yo = train_test_split(xtr,ytr,test_size=0.5)

In [ ]:
for i in range(len(xtr)):
    forward_propogate(network,xtr[i])
    error = ytr[i]-network[-1][0].output
    back_propogate(network,error)

In [ ]:
def feed_path(network,path,inp,out,f = 0):
    m = inp
    for i in range(len(path)-1):
        m = sigmoid(network[i][path[i]].weight[path[i+1]]*m)
    if f==1:
        return sigmoid(network[-1][0].weight[0]*m)
    return out-sigmoid(network[-1][0].weight[0]*m)

In [ ]:
def genetic(network,data,output):
    paths = select_paths(network,n=100)
    for i in range(len(data)):
        e = []
        for j in paths:
            e.append(feed_path(network,j,data[i],output[i]))
        e.sort()
        m = crossover(e[0],e[1])
        paths[-1] = m[0]
        paths[-2] = m[1]
        m = mutate(network,paths[random.randint()%len(paths)])
        if m != -1:
            paths[-3] = m
    return paths

In [14]:
paths=genetic(network,xo,yo)

NameError: name 'genetic' is not defined

In [ ]:
for i in range(len(xt)):
    l=[]
    for j in paths:
        l.append(feed_path(network,j,xt[i],yt[i],f = 1))
    out = sum(l)
    out